In [ ]:
# Interactive Figure
# %matplotlib ipympl 
# Non interactive
%matplotlib inline 

In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt
from qualang_tools.units import unit
u = unit(coerce_to_integer=True)
from qm.qua import *
from qm import QuantumMachinesManager
from qm import SimulationConfig
from qualang_tools.results import fetching_tool
from qualang_tools.loops import from_array
import config_00 as config
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def addjob(qmprog, qm):
    # Add a QUA program to the OPX queue, which compiles it and executes it
    job = qm.queue.add(qmprog)
    # Wait for job to be loaded
    while job.status=="loading":
        print("Job is loading...")
        time.sleep(0.1)
    # Wait until job is running
    time.sleep(0.1)
    while job.status=="pending":
        q = job.position_in_queue()
        if q>0:
            print("Position in queue",q,end='\r')
        time.sleep(0.1)
    job=job.wait_for_execution()
    print("\nJob is running")
    return job

In [ ]:
# Connect to the cluster (run only once)
import QM_cluster
qmm = QuantumMachinesManager(host=QM_cluster.QM_Router_IP, cluster_name=QM_cluster.cluster_name)

# Get running QM instance

In [ ]:
# Get the QM reference (rerun every time the config is changed)
qm_list =  qmm.list_open_qms()
qm = qmm.get_qm(qm_list[0])
print(f"Connected to {qm.id}")

# Send Pulses and Demodulate them

In [ ]:
# This block is using QUA directives which are compiled to the FPGA
chunk_size = 5 # 20ns
IQ_size = config.readout_len//(chunk_size*4)
with program() as prog:
    # Variable declaration
    m = declare(int)
    I = declare(fixed, size=IQ_size)
    Q = declare(fixed, size=IQ_size)
    I_st = declare_stream()
    Q_st = declare_stream()
    # Start measurement
    measure("readout", "scope",
    demod.sliced("cos", I, chunk_size, "out1"),
    demod.sliced("minus_sin", Q, chunk_size, "out1"),
            )
    # Pulse sequence
    play('pulse','qubit',duration=100*u.ns)
    wait(200*u.ns)
    play('pulse','qubit',duration=100*u.ns)
                       
    with for_(m, 0, m < IQ_size, m + 1): 
        save(I[m], I_st)
        save(Q[m], Q_st)

    # Select data to be sent to the client 
    with stream_processing():
        I_st.buffer(IQ_size).save('I')
        Q_st.buffer(IQ_size).save('Q')

# Run the code and fetch results
job = addjob(prog, qm)
res = fetching_tool(job, ['I','Q'])
I, Q = res.fetch_all()
print('Job done')

# Plot
t = np.arange(IQ_size)*chunk_size*4
fig,ax=plt.subplots()
ax.plot(t,I,t,Q)

# Practice
- Link to the Quantum Machine documentation [https://docs.quantum-machines.co/latest/](https://docs.quantum-machines.co/latest/)
- Read the doc about [demodulation](https://docs.quantum-machines.co/latest/docs/Guides/demod/)

## Exercise 1: IQ rotation
- Dephase the second pulse by $\pi/2$ using a [frame_rotation](https://docs.quantum-machines.co/latest/docs/API_references/qua/dsl_main/#qm.qua._dsl.frame_rotation) command. Change the delay between the pulses, what do you observe?
- Try different dephasings in order to produce two pulses with negative $I$ values, one pulse with positive $I$ and the second with positive $Q$, and so on...   

In [ ]:
# This block is using QUA directives which are compiled to the FPGA
chunk_size = 5 # 20ns
IQ_size = config.readout_len//(chunk_size*4)
with program() as prog:
    # Variable declaration
    m = declare(int)
    I = declare(fixed, size=IQ_size)
    Q = declare(fixed, size=IQ_size)
    I_st = declare_stream()
    Q_st = declare_stream()
    # Start measurement
    measure("readout", "scope",
    demod.sliced("cos", I, chunk_size, "out1"),
    demod.sliced("minus_sin", Q, chunk_size, "out1"),
            )
    # Pulse sequence
    play('pulse','qubit',duration=100*u.ns)
    frame_rotation_2pi(...)
    wait(200*u.ns)
    play('pulse','qubit',duration=100*u.ns)
                       
    with for_(m, 0, m < IQ_size, m + 1): 
        save(I[m], I_st)
        save(Q[m], Q_st)

    # Select data to be sent to the client 
    with stream_processing():
        I_st.buffer(IQ_size).save('I')
        Q_st.buffer(IQ_size).save('Q')

# Run the code and fetch results
job = addjob(prog, qm)
res = fetching_tool(job, ['I','Q'])
I, Q = res.fetch_all()
print('Job done')

# Plot
t = np.arange(IQ_size)*chunk_size*4
fig,ax=plt.subplots()
ax.plot(t,I,t,Q)

## Exercise 2: Averaging 
- Go back to the previous exercise and reduce the amplitude of the two pulses by a factor 1000 using the `amp` command inside the `play`. Do you still see the pulses?
- Averaging small signals is a common problem in circuit QED. Modify the code below to repeat the whole sequence 10000 times in a for loop. In order to average over the loop repetitions, the stream processing block is changed to:
  
      with stream_processing():
        I_st.buffer(IQ_size).average().save('I')
        Q_st.buffer(IQ_size).average().save('Q')

  (You can read the doc on [stream buffering and averaging](https://docs.quantum-machines.co/latest/docs/Guides/stream_proc/#using-stream-operators))
- Replace one pulse by a gaussian pulse (remove the duration keyword) and observe the result 

In [ ]:
# This block is using QUA directives which are compiled to the FPGA
chunk_size = 5 # 20ns
IQ_size = config.readout_len//(chunk_size*4)
with program() as prog:
    # Variable declaration
    m = declare(int)
    n = declare(int)
    I = declare(fixed, size=IQ_size)
    Q = declare(fixed, size=IQ_size)
    I_st = declare_stream()
    Q_st = declare_stream()
    # Repeat measurement
    with for_(n, 0, n < 1, n + 1): 
        measure("readout", "scope",
        demod.sliced("cos", I, chunk_size, "out1"),
        demod.sliced("minus_sin", Q, chunk_size, "out1"),
                )
        # Pulse sequence
        play('pulse'*amp(0.001),'qubit',duration=100*u.ns)
        wait(200*u.ns)
        play('pulse'*amp(0.001),'qubit',duration=100*u.ns)
                       
        with for_(m, 0, m < IQ_size, m + 1): 
            save(I[m], I_st)
            save(Q[m], Q_st)

    # Select data to be sent to the client 
    with stream_processing():
        I_st.buffer(IQ_size).average().save('I')
        Q_st.buffer(IQ_size).average().save('Q')

# Run the code and fetch results
job = addjob(prog, qm)
res = fetching_tool(job, ['I','Q'])
I, Q = res.fetch_all()
print('Job done')

# Plot
t = np.arange(IQ_size)*chunk_size*4
fig,ax=plt.subplots()
ax.plot(t,I,t,Q)